# Install Dependencies

In [1]:
%pip install tensorflow
%pip install tensorflow-gpu
%pip install tensorflow-hub
%pip install opencv-python
%pip install matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow-gpu (from versions: none)
ERROR: No matching distribution found for tensorflow-gpu

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import Dependencies

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np
import os

In [3]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[]


# Load Movenet Model

In [4]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# Make Detections

In [5]:
# Fucntion that draws points of the joints on the screen
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [6]:
# object that mapes the connections between each point drawn on the screen
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
# Function that draws the edges connecting the points drwan by draw_keypoints fucntion
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [8]:
# Function to loop through each person detected and render with maximum number 6 persons
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

# Open Camera and Render Skeleton

In [9]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0),192,256)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints 
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.35)
    
    cv2.imshow('Human Gait Identification', frame)
    
    if cv2.waitKey(10) & 0xFF== 27:
        break
cap.release()
cv2.destroyAllWindows()

# Folder Setup for Data Collection

In [10]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Temp_Dataset') 

# Persons that we try to detect
persons = np.array(['Lance','Seif'])

#Very person with 4 different view
views = np.array(['Front','Back','Left','Right'])

# 10 videos worth of data
no_sequences = 20

# Videos are going to be 30 frames in length
sequence_length = 60

In [11]:
for person in persons:
    for view in views:
        for sequence in range(no_sequences):
            try: 
                os.makedirs(os.path.join(DATA_PATH, person, view, str(sequence)))
            except:
                pass

# Dataset Collection

In [12]:
cap = cv2.VideoCapture(0)
for person in persons:
    for view in views:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()
                    
                # Resize image
                image = frame.copy()
                image = tf.image.resize_with_pad(tf.expand_dims(image, axis=0),192,256)
                input_img = tf.cast(image, dtype=tf.int32)
                    
                # Detection section
                results = movenet(input_img)
                extracted_keypoints = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
                    
                # Render keypoints 
                loop_through_people(frame, extracted_keypoints, EDGES, 0.35)

                if frame_num == 0: 
                    cv2.putText(frame, 'Starting {} Collection'.format(person), (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
                    cv2.putText(frame, 'Collecting frames for {} with {} view, Video Number {}'.format(person, view, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(3000)
                else: 
                    cv2.putText(frame, 'Capturing'.format(person), (120,200), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 1, cv2.LINE_AA)
                    cv2.putText(frame, 'Collecting frames for {} with {} view, Video Number {}'.format(person, view, sequence), (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                        
                # NEW Export keypoints
                keypoints = extracted_keypoints
                npy_path = os.path.join(DATA_PATH, person, view, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                    
                cv2.imshow('Human Gait Identification', frame)
        
                if cv2.waitKey(10) & 0xFF== 27:
                    break
cap.release()
cv2.destroyAllWindows()

# CNN Model

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

ModuleNotFoundError: No module named 'sklearn'

In [15]:
# Assuming you have a list of persons
persons = ['Lance', 'Seif']  # Add more names as needed

# Load data and labels
labels = []
data = []

# Iterate over persons
for person in persons:
    # Iterate over views
    for view in ['Front', 'Back', 'Left', 'Right']:  # Add more views as needed
        # Iterate over sequences
        for sequence in range(20):
            # Load .npy file for each sequence
            file_path = f'Custom_Gait_Dataset/{person}/{view}/{sequence}.npy'
            sequence_data = np.load(file_path)
            
            # Assuming each .npy file has a shape (90, 17, 4)
            # where 90 is the number of frames, 17 is the number of joints, and 4 represents (X, Y, Z, visibility)
            
            # Reshape data to (90, 17, 4, 1) for Conv2D input
            sequence_data = sequence_data[:, :, :, np.newaxis]

            labels.append(person)
            data.append(sequence_data)

# Convert lists to numpy arrays
data = np.array(data)
labels = np.array(labels)

# One-hot encode the labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)
labels_one_hot = np_utils.to_categorical(labels_encoded)

# Preprocess the data
data = data / 255.0  # Assuming pixel values are between 0 and 255

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels_one_hot, test_size=0.2, random_state=42)

# Build the CNN Model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(90, 17, 4, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(persons), activation='softmax'))  # Output layer with the number of persons

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy[1]*100:.2f}%')

FileNotFoundError: [Errno 2] No such file or directory: 'Custom_Gait_Dataset/Lance/Front/0.npy'